In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# importing the libararies

In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import pandas_datareader as data_reader

from tqdm import tqdm_notebook , tqdm
from collections import deque

# building the AI trader network

In [3]:
class AI_Trader():

  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell

    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()

  def model_builder(self):

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))

    model.add(tf.keras.layers.Dense(units=64, activation='relu'))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))

    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))

    return model

  def trade(self, state):

    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)
    return np.argmax(actions[0])


  def batch_train(self, batch_size):

    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target, epochs=1, verbose=0)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay



# data set preprocessing

## sigmoid

In [4]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

## price format function

In [5]:
def stock_price_format(n):
    if n < 0 :
        return '- $ {:2f}'.format(abs(n))
    else :
        return '$ {:2f}'.format(abs(n))

# dataset loader

In [6]:
def dataset_loader(stock_name):
    data = yf.Ticker(stock_name)
    dataset = data.history(period="1d")
    close = dataset['Close']
    return close


In [7]:
import yfinance as yf

In [8]:
stock_name = 'AAPL'
data = yf.Ticker(stock_name)
dataset_for_example = data.history(period="1d")
dataset_for_example


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-15 00:00:00-04:00,178.880005,179.470001,177.380005,177.869995,26557921,0.0,0.0


# state creator

In [9]:
def state_creator(data, timestep, window_size):

  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

  return np.array([state])

# loading a dataset

In [10]:
stock_name = 'AAPL'
data = dataset_loader(stock_name)

data

Date
2023-08-15 00:00:00-04:00    177.880005
Name: Close, dtype: float64

# Training the AI trader

## setting the hyper parameters

In [11]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1


## defining the trader model

In [12]:
trader = AI_Trader(window_size)

In [13]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# training loop ( # this training process will take time )

In [14]:

 for episode in range(1, episodes + 1):

  print("Episode: {}/{}".format(episode, episodes))

  state = state_creator(data, 0, window_size + 1)

  total_profit = 0
  trader.inventory = []

  for t in tqdm(range(data_samples)):

    action = trader.trade(state)

    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0

    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stock_price_format(data[t]))

    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )

    if t == data_samples - 1:
      done = True
    else:
      done = False

    trader.memory.append((state, action, reward, next_state, done))

    state = next_state

    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

Episode: 1/1000


0it [00:00, ?it/s]


Episode: 2/1000


0it [00:00, ?it/s]


Episode: 3/1000


...

0it [00:00, ?it/s]


Episode: 814/1000


0it [00:00, ?it/s]


Episode: 815/1000


0it [00:00, ?it/s]
